# Goal:

# Imports

In [1]:
import pandas as pd
import geopandas as gpd
import googlemaps
from shapely.geometry import Point
import os
import numpy as np
import re
from textwrap import shorten
from bs4 import BeautifulSoup

## Data read-in

In [2]:
df = pd.read_csv("Wynwood Live Local Act projects - Sheet1.csv")
df

,Project name,Developer,Address,Google coordinates,Size,Where does it stand with approvals?
0,NaN,Bazbaz Development,"2110, 2118 and 2134 North Miami Avenue, as wel...","25.79729007306927, -80.19535522477574","48-story, 544-unit partment tower",NaN
1,NaN,Hidrock Properties and Robert Finvarb of Rober...,2534 North Miami Avenue,"25.801424299938954, -80.19525048627189",39-story tower with 336 apartments,NaN
2,NaN,"Miami Court Holdings, led by Joseph Cohen","2100 Northwest Miami Court, as well as 2101 an...","25.797210654548568, -80.19636345999774","19-story, 401-unit apartment tower",NaN
3,NaN,David Sedaghati’s Ultimate Equity,"520, 524, 530, 540, 550, 570, 580 and 590 NOrt...","25.800676430497518, -80.20452240321376",A 25-story complex with 996 apartments and 693...,NaN
4,NaN,Clara Homes,2601 Northwest Sixth Avenue,"25.80109443927699, -80.2050117352153",An 18-story to 20-story building with about 15...,NaN


## Geocode

In [3]:
# df['full_address'] = df['address'] + ' IL'

In [4]:
# %store -r google_maps_API_Key
# gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [5]:
# # Define the geocode function
# def geocode(add):
#     g = gmaps_key.geocode(add)
#     if g:
#         lat = g[0]["geometry"]["location"]["lat"]
#         lng = g[0]["geometry"]["location"]["lng"]
#         return (lat, lng)
#     else:
#         return None

# # Apply geocoding to the 'Full_Address' column and store the results in the 'geocoded' column
# df['geocoded'] = df['full_address'].apply(geocode)

# # Split 'geocoded' into 'lat' and 'lon', handling None values
# df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x is None else x).apply(pd.Series)

# # Convert 'lat' and 'lon' to floats
# df['lat'] = df['lat'].astype(float)
# df['lon'] = df['lon'].astype(float)

# # Create the 'geometry' column with Point objects
# df['geometry'] = df.apply(lambda row: Point(row['lon'], row['lat']) if pd.notnull(row['lat']) and pd.notnull(row['lon']) else None, axis=1)

# # Drop rows where geometry is None
# df = df.dropna(subset=['geometry'])

# # Create a GeoDataFrame
# gdf = gpd.GeoDataFrame(df, geometry='geometry')

# # Set CRS (Coordinate Reference System) to WGS 84 (EPSG:4326)
# gdf.set_crs(epsg=4326, inplace=True)


In [6]:
df

,Project name,Developer,Address,Google coordinates,Size,Where does it stand with approvals?
0,NaN,Bazbaz Development,"2110, 2118 and 2134 North Miami Avenue, as wel...","25.79729007306927, -80.19535522477574","48-story, 544-unit partment tower",NaN
1,NaN,Hidrock Properties and Robert Finvarb of Rober...,2534 North Miami Avenue,"25.801424299938954, -80.19525048627189",39-story tower with 336 apartments,NaN
2,NaN,"Miami Court Holdings, led by Joseph Cohen","2100 Northwest Miami Court, as well as 2101 an...","25.797210654548568, -80.19636345999774","19-story, 401-unit apartment tower",NaN
3,NaN,David Sedaghati’s Ultimate Equity,"520, 524, 530, 540, 550, 570, 580 and 590 NOrt...","25.800676430497518, -80.20452240321376",A 25-story complex with 996 apartments and 693...,NaN
4,NaN,Clara Homes,2601 Northwest Sixth Avenue,"25.80109443927699, -80.2050117352153",An 18-story to 20-story building with about 15...,NaN


In [7]:
import ast
import geopandas as gpd
from shapely.geometry import Point

# Convert string representation of tuples to actual tuples
df['Google coordinates'] = df['Google coordinates'].apply(lambda x: ast.literal_eval(x))

# Create the geometry column (longitude, latitude)
df['geometry'] = df['Google coordinates'].apply(lambda x: Point((x[1], x[0])))  # Reorder to (longitude, latitude)

# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set the CRS to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# Now gdf is a GeoDataFrame!


,Project name,Developer,Address,Google coordinates,Size,Where does it stand with approvals?,geometry
0,NaN,Bazbaz Development,"2110, 2118 and 2134 North Miami Avenue, as wel...","(25.79729007306927, -80.19535522477574)","48-story, 544-unit partment tower",NaN,POINT (-80.19536 25.79729)
1,NaN,Hidrock Properties and Robert Finvarb of Rober...,2534 North Miami Avenue,"(25.801424299938954, -80.19525048627189)",39-story tower with 336 apartments,NaN,POINT (-80.19525 25.80142)
2,NaN,"Miami Court Holdings, led by Joseph Cohen","2100 Northwest Miami Court, as well as 2101 an...","(25.797210654548568, -80.19636345999774)","19-story, 401-unit apartment tower",NaN,POINT (-80.19636 25.79721)
3,NaN,David Sedaghati’s Ultimate Equity,"520, 524, 530, 540, 550, 570, 580 and 590 NOrt...","(25.800676430497518, -80.20452240321376)",A 25-story complex with 996 apartments and 693...,NaN,POINT (-80.20452 25.80068)
4,NaN,Clara Homes,2601 Northwest Sixth Avenue,"(25.80109443927699, -80.2050117352153)",An 18-story to 20-story building with about 15...,NaN,POINT (-80.20501 25.80109)


In [8]:
# Convert all data to strings if possible, or remove complex structures
for col in gdf.columns:
    if gdf[col].dtype == object:
        try:
            gdf[col] = gdf[col].astype(str)
        except Exception as e:
            print(f"Could not convert column {col}: {e}")
            gdf.drop(columns=[col], inplace=True)


In [9]:
gdf.to_file('Live_local.geojson', driver='GeoJSON')

In [10]:
gdf.columns

Index(['Project name', 'Developer', 'Address', 'Google coordinates', 'Size',
       'Where does it stand with approvals? ', 'geometry'],
      dtype='object')

In [11]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/LiveLocalAct
